In [1]:
import os
from dotenv import load_dotenv
from pathlib import Path
from pyspark.sql import SparkSession
from typing import Union

In [2]:
def get_minio_credentials(path_to_env_file: Union[str, Path] = "../config/minio.env") -> str:
    """This function returns login and password for root user of minio server, getting those from 'minio.env' file.
    'minio.env' should have MINIO_ROOT_USER and MINIO_ROOT_PASSWORD variables. If there are no such variables, asks user to provide those via input.
    Args:
        path_to_env_file: either string or pathlib.Path object leading to minio.env file."""
    load_dotenv(path_to_env_file)
    MINIO_ROOT_USER = os.getenv("MINIO_ROOT_USER")
    MINIO_ROOT_PASSWORD = os.getenv("MINIO_ROOT_PASSWORD")
    if (MINIO_ROOT_USER and MINIO_ROOT_PASSWORD):
        return MINIO_ROOT_USER, MINIO_ROOT_PASSWORD 
    else:
        print(f"There are no MINIO_ROOT_USER and/or MINIO_ROOT_PASSWORD variables in {path_to_env_file}")

login_minio, password_minio = get_minio_credentials()

spark = (
    SparkSession.builder
    .appName("bronze_to_silver")
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
    .config("spark.hadoop.fs.s3a.access.key", login_minio)
    .config("spark.hadoop.fs.s3a.secret.key", password_minio)
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    #.config("spark.driver.extraJavaOptions", "-Djava.security.manager") # to solve JDK 23+ compatibility problem
    .getOrCreate()
)


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/13 15:52:15 WARN Utils: Your hostname, ubuntu-home, resolves to a loopback address: 127.0.1.1; using 192.168.0.29 instead (on interface enp3s0)
26/01/13 15:52:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/13 15:52:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [11]:
s3_file_path = "s3a://airflow.learn/2026-01-13 08:37:19.193383+00:00"

In [12]:
bronze_df = spark.read\
    .format("binaryFile")\
    .load(s3_file_path)

26/01/13 16:14:14 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: s3a://airflow.learn/2026-01-13 08:37:19.193383+00:00.
java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2737)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3569)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3612)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:172)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3716)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3667)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:557)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:366)
	at org.apache.spark.sql.execution.streaming.sinks.FileStreamSink$.hasMetadata(FileStreamSink.scala:57)
	at org.apache.spark.sql.execution.dataso

Py4JJavaError: An error occurred while calling o57.load.
: java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2737)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3569)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3612)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:172)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3716)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3667)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:557)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:366)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$1(DataSource.scala:782)
	at scala.collection.immutable.List.map(List.scala:236)
	at scala.collection.immutable.List.map(List.scala:79)
	at org.apache.spark.sql.execution

In [10]:
bronze_df.show()

NameError: name 'bronze_df' is not defined

In [13]:
pip uninstall pyspark

Found existing installation: pyspark 4.1.1
Uninstalling pyspark-4.1.1:
  Would remove:
    /home/heinz/miniconda3/envs/stonks/bin/beeline
    /home/heinz/miniconda3/envs/stonks/bin/beeline.cmd
    /home/heinz/miniconda3/envs/stonks/bin/docker-image-tool.sh
    /home/heinz/miniconda3/envs/stonks/bin/find-spark-home
    /home/heinz/miniconda3/envs/stonks/bin/find-spark-home.cmd
    /home/heinz/miniconda3/envs/stonks/bin/find_spark_home.py
    /home/heinz/miniconda3/envs/stonks/bin/load-spark-env.cmd
    /home/heinz/miniconda3/envs/stonks/bin/load-spark-env.sh
    /home/heinz/miniconda3/envs/stonks/bin/pyspark
    /home/heinz/miniconda3/envs/stonks/bin/pyspark.cmd
    /home/heinz/miniconda3/envs/stonks/bin/pyspark2.cmd
    /home/heinz/miniconda3/envs/stonks/bin/run-example
    /home/heinz/miniconda3/envs/stonks/bin/run-example.cmd
    /home/heinz/miniconda3/envs/stonks/bin/spark-class
    /home/heinz/miniconda3/envs/stonks/bin/spark-class.cmd
    /home/heinz/miniconda3/envs/stonks/bin/spa

In [14]:
pip uninstall pyspark -Y


Usage:   
  /home/heinz/miniconda3/envs/stonks/bin/python -m pip uninstall [options] <package> ...
  /home/heinz/miniconda3/envs/stonks/bin/python -m pip uninstall [options] -r <requirements file> ...

no such option: -Y
Note: you may need to restart the kernel to use updated packages.
